In [ ]:
suppressMessages(library(tidyverse))
suppressMessages(library(reshape2))
suppressMessages(library(stats))
suppressMessages(library(pROC))
library(plotly)
library(rgl)
library(circlize)
library(caret)

source("../theme_ggplot_prevail.R")
source("../global_variables.R")

gene_list = read.delim("../0_DATA/gencode.biotype.name.key.tsv")


Attaching package: ‘plotly’


The following object is masked from ‘package:ggplot2’:

    last_plot


The following object is masked from ‘package:stats’:

    filter


The following object is masked from ‘package:graphics’:

    layout


This build of rgl does not include OpenGL functions.  Use
 rglwidget() to display results, e.g. via options(rgl.printRglwidget = TRUE).

circlize version 0.4.15
CRAN page: https://cran.r-project.org/package=circlize
Github page: https://github.com/jokergoo/circlize
Documentation: https://jokergoo.github.io/circlize_book/book/

If you use it in published research, please cite:
Gu, Z. circlize implements and enhances circular visualization
  in R. Bioinformatics 2014.

This message can be suppressed by:
  suppressPackageStartupMessages(library(circlize))


Loading required package: lattice


Attaching package: ‘caret’


The following object is masked from ‘package:purrr’:

    lift




In [2]:
counts_train_path = "../0_DATA/counts_train.7030.csv"
counts_test_path = "../0_DATA/counts_test.7030.csv"

counts_vst_train_path = "../0_DATA/counts_train_vst.7030.csv"
counts_vst_test_path = "../0_DATA/counts_test_vst.7030.csv"

meta_train_path = "../0_DATA/meta_data_train.7030.csv"
meta_test_path = "../0_DATA/meta_data_test.7030.csv"

counts_train = read.csv(counts_train_path, row.names = 1)
counts_test = read.csv(counts_test_path, row.names = 1)
counts_train_vst = read.csv(counts_vst_train_path, row.names = 1)
counts_test_vst = read.csv(counts_vst_test_path, row.names = 1)
meta_train = read.csv(meta_train_path) %>% mutate(cohort = "train")
meta_test = read.csv(meta_test_path) %>% mutate(cohort = "test")

meta_all <- rbind(meta_train, meta_test)

---
## Plots for overview

In [3]:
chord_mtx = matrix(rep(1,4*4),ncol=4)
colnames(chord_mtx) <- c("bacterial_infection",  "KD", "MISC", "viral_infection")
rownames(chord_mtx) <- c("bacterial_infection",  "KD", "MISC", "viral_infection")


WIDTH = 2
HEIGHT = 2

pdf(file = paste0("./plots/ALL-ONE_SUMMARY_chord.pdf"),
        width=WIDTH,height=HEIGHT, paper="special", bg="white",
        fonts="Helvetica", colormodel = "srgb", pointsize=6, useDingbats = FALSE)

chordDiagram(chord_mtx, 
    transparency = 0.5,
    # grid.col = c("red", "blue", "purple","pink", "orange", "green", "steelblue1"),
    grid.col = c(INFLAMCAT_FILL_KEY[['bacterial_infection']], 
                #  INFLAMCAT_FILL_KEY[['Healthy']], 
                #  INFLAMCAT_FILL_KEY[['other']], 
                 INFLAMCAT_FILL_KEY[['viral_infection']], 
                 INFLAMCAT_FILL_KEY[['MISC']], 
                 INFLAMCAT_FILL_KEY[['KD']]
                 ),
    # annotationTrack = c("name","grid"),
    annotationTrack = c("grid")
    )

dev.off()

library("VennDiagram")
library(eulerr)


# create a list of sets

WIDTH = 1
HEIGHT = 1

pdf(file = paste0("./plots/ALL-ONE_SUMMARY_venn.pdf"),
                width=WIDTH,height=HEIGHT, paper="special", bg="white",
                fonts="Helvetica", colormodel = "srgb", pointsize=6, useDingbats = FALSE)

set_list <- list("Set 1" = c(1, 2, 3, 4),
                                "Set 2" = c(3, 4, 5, 6))
plot(euler(set_list),
     fills = c("#CD4254", "#FBE77C"),
     edges = T,
     labels = list(cex = 0),
     quantities = list(cex = 0))

dev.off()



png 
  2

Loading required package: grid

Loading required package: futile.logger



png 
  2

---
## Wrangle outputs of pairwise DAA

#### Functions

In [4]:
get_sig <- function(C,rds,grp){

    ## get sig df & add comparison
    sig_df <- rds[[C]]$res_df %>% 
        dplyr::rename(gene_id = Row.names) %>% 
        mutate(comp = C) %>%
        select(comp, gene_id, gene_name, gene_type, padj, log2FoldChange, baseMean, gene_name)

    ## change log2FC direction to match group of interest
    if(grp == unlist(strsplit(C,"<>"))[2]){
        sig_df <- sig_df %>% mutate(log2FoldChange = -1 * log2FoldChange)
    }

    return(sig_df)
}


get_overlapping_genes <- function(GRP,daa_rds,comparisons){

    ## get all relevant comparisons
    grp_comps <- comparisons[grepl(GRP,comparisons)]

    ## extract all significant genes and make data frame
    all_sig <- lapply(grp_comps, get_sig, daa_rds,GRP)
    all_sig_df <- do.call("rbind",all_sig) %>% 
        data.frame()

    ## sumarize to count occurrences
    sig_counts <- all_sig_df %>% 
        filter(padj < 0.05) %>% 
        group_by(gene_id,gene_name) %>% 
        dplyr::count() %>% dplyr::rename(occurrences = n) %>% 
        arrange(desc(occurrences))

    ## counts occurrences for plot
    num_occur <- sig_counts  %>% group_by(occurrences) %>% dplyr::count()

    return(list("all_sig_df" = all_sig_df, "sig_counts" = sig_counts, "num_occur" = num_occur))

}

#### Loop over outputs

In [5]:
## load all DAA results
daa_rds <- readRDS("./output/pairwise_DAA.train.rds")

# ## get all pairwise comparisons
# all_groups = unique(meta_all$inflam_cat)
# print(all_groups)
# cat("\n")
# comparisons = daa_rds %>% names
# print(comparisons)

# ## Iterate over all gorups
# all_outputs <- list()
# for (GRP in all_groups){

#     ## get overlapping data frames
#     all_out <- get_overlapping_genes(GRP,daa_rds,comparisons)

#     ## save to output
#     all_outputs[[GRP]] = all_out

#     ## make plot
#     options(repr.plot.height = 4, repr.plot.width = 6)
#     plt <- all_out[['num_occur']] %>% 
#         ggplot(aes(x=occurrences, y = n))+
#         geom_bar(stat="identity",width = 1, fill="forest green")+   #forest green   red  blue
#         xlim(0,7)+
#         theme_prevail()+
#         labs(y="genes", title = GRP)+
#         theme(axis.text = element_text(size = 12),
#             axis.title = element_text(size = 12),
#             title = element_text(size = 12))

#     # print(plt)
#         }

# all_outputs %>% saveRDS("./output/occur.rds")


all_outputs = readRDS("./output/occur.rds")

### save outputs

In [6]:
# daa_rds <- readRDS("./output/pairwise_DAA.train.rds")


### save outputs
comparisons <- daa_rds %>% names


for (COMP in comparisons){
    res = daa_rds[[COMP]]$res_df %>% dplyr::rename(gene_id = Row.names)
    res %>% write.csv(paste0("./output/Supp_",gsub("<>","-",COMP),"_ML-DESEQ.csv"),row.names = F,quote=FALSE)
    }

---
# Individual Models

### Check DEA

In [3]:
all_outputs = readRDS("./output/occur.rds")

PVAL_THRESH = 0.05
LOG2FC_THRESH = 0.25
BASEMEAN_THRESH = 50

ALL_MODS = all_res_mdf %>% pull(model) %>% unique()

MOD = ALL_MODS[1]

for (MOD in ALL_MODS){

    res = all_outputs[[gsub("<>.*","",MOD)]][["all_sig_df"]] %>% filter(grepl(gsub(".*<>","",MOD),comp))
    NUM = res %>% 
        filter(padj < PVAL_THRESH) %>% 
        filter(abs(log2FoldChange) > LOG2FC_THRESH) %>%
        filter(baseMean > BASEMEAN_THRESH) %>% 
        nrow()

    cat(paste(MOD," --> ",as.character(NUM),"\n"))

    # res %>% 
    #     filter(padj < PADJ_FILT) %>% 
    #     filter(abs(log2FoldChange) > L2FC_FILT) %>%
    #     filter(baseMean > BM_FILT) %>%
    #     group_by(gene_id,gene_name) %>% 
    #     summarise(n = n(), mean_lg = mean(abs(log2FoldChange)))%>%           ## group by and count occurances
    #     arrange(desc(n)) %>% head() %>% print()
}



ERROR: Error in pull(., model): object 'all_res_mdf' not found


### Load outputs

In [3]:
res_path = "./ML_split70-30/1_all/output/"

all_comps = c(
    "KD<>bacterial_infection",
    "KD<>viral_infection",
    "MISC<>bacterial_infection",
    "MISC<>viral_infection",
    "MISC<>KD",
    "viral_infection<>bacterial_infection"
)

ALG = "GLMNETLasso"

all_mod <- list()
all_res <- list()
all_youden <- list()

for (GRP in all_comps){
    grp_mod = readRDS(paste0(res_path,GRP,"/",ALG,"/",GRP,".",ALG,".rds"))
    grp_res <- rbind(grp_mod[['meta_data_train']], grp_mod[['meta_data_test']]) %>% 
        select(cfrna_id, classifier_score) %>%
        mutate(model = GRP)
    grp_youden = read.delim(paste0(res_path,GRP,"/",ALG,"/",GRP,".",ALG,".train.txt"))["youden",]

    all_mod[[GRP]] <- grp_mod
    all_res[[GRP]] <- grp_res
    all_youden[[GRP]] <- grp_youden

}

all_res <- do.call(rbind,all_res)


### Create master data frame
all_res_mdf <- merge(all_res, meta_all, by = "cfrna_id") %>%
    select(cfrna_id,inflam_cat, cohort, model, classifier_score) %>%
    mutate(cohort = factor(cohort, levels = c("train","test")))%>%
    mutate(inflam_cat = factor(inflam_cat, levels = c("MISC","KD","viral_infection","bacterial_infection","Healthy","other")))%>% 
    mutate(model = factor(model, levels = all_comps))

    

### Plot scores

In [33]:
all_res_mdf %>% filter(model == "MISC<>viral_infection") %>% filter(cohort == "test") %>% pull(inflam_cat)

[1] viral_infection viral_infection viral_infection viral_infection
 [5] MISC            MISC            MISC            MISC           
 [9] MISC            MISC            MISC            KD             
[13] viral_infection MISC            MISC            viral_infection
[17] viral_infection viral_infection viral_infection viral_infection
[21] viral_infection viral_infection viral_infection viral_infection
[25] viral_infection viral_infection viral_infection viral_infection
[29] viral_infection viral_infection viral_infection viral_infection
[33] MISC            MISC            MISC            MISC           
[37] viral_infection MISC            viral_infection MISC           
[41] viral_infection viral_infection viral_infection viral_infection
[45] viral_infection viral_infection MISC            MISC           
[49] MISC            MISC            viral_infection MISC           
[53] MISC            MISC            MISC            MISC           
[57] MISC            MISC            MISC            MISC           
[61] MISC            viral_infection viral_infection viral_infection
[65] MISC            MISC           
Levels: MISC KD viral_infection bacterial_infection Healthy other

In [34]:
### Create train / test plots for all but bacterial

ALL_MODS = all_res_mdf %>% pull(model) %>% unique()
# ALL_MODS = ALL_MODS[!grepl("bacterial_infection", ALL_MODS)]

for (MOD in ALL_MODS){

    set.seed(42)

    boxplt = all_res_mdf %>% 
        mutate(inflam_cat = factor(inflam_cat, levels = names(INFLAMCAT_FILL_KEY))) %>% 
        filter(inflam_cat %in% unlist(strsplit(MOD,"<>"))) %>%
        filter(model == MOD) %>%
        ggplot(aes(x=inflam_cat, y = classifier_score, fill = inflam_cat)) +
        geom_hline(yintercept = all_youden[[MOD]], linetype = "dashed", alpha = 0.75)+
        # geom_violin(color = NA, alpha = 0.9, width = 10) +
        geom_boxplot(outlier.shape = NA, width = 0.5, color = "black")+
        geom_point(size = 0.5, alpha = 0.75, shape= 21, stroke = .2, position = position_jitter(width= 0.2, height = 0))+
        theme_prevail() +
        theme(
            # strip.background = element_rect(color="black", fill="white", size=.75, linetype="solid"),
            strip.background = element_blank(),
            # strip.text = element_text(size = 8),
            strip.text = element_blank(),

            panel.spacing = unit(1.5, "lines"),

            axis.title.x = element_blank(),
        #     axis.text.x = element_text(size = 6, angle = 45, hjust = 1, vjust = 1),
            axis.text.x = element_blank(),
            axis.title.y = element_blank(), 
            plot.title = element_text(hjust = 0.5, size = 8)) +
        scale_fill_manual(values = INFLAMCAT_FILL_KEY)+
        # facet_wrap(~cohort, ncol = 2)+
        facet_wrap(~cohort, ncol = 1)+
        scale_y_continuous(limits = c(0,1), breaks = seq(0,1,0.25), labels = c("0.00","0.25","0.50","0.75","1.00"))


    # WIDTH = 2.7
    # HEIGHT = 1.3

    WIDTH = 0.8
    HEIGHT = 1.1

    pdf(file = paste0("./plots/ONE-ONE_class_scores.",MOD,".pdf"),
            width=WIDTH,height=HEIGHT, paper="special", bg="white",
            fonts="Helvetica", colormodel = "srgb", pointsize=6, useDingbats = FALSE)

    print(boxplt)
    dev.off()
}


### AUC plots

In [8]:
### Create train / test AUC plots for all but bacterial

ALL_MODS = all_res_mdf %>% pull(model) %>% unique()


all_mod_metrics <- list()

for (MOD in ALL_MODS){
    mod_data = all_mod[[MOD]]
    meta_data_train <- mod_data$meta_data_train
    meta_data_test <- mod_data$meta_data_test

    ## GET TPR + FPR

    roc_train <- suppressMessages(roc(meta_data_train$group, meta_data_train$classifier_score,
                                percent=TRUE,
    #                             ci=TRUE, 
                                boot.n=10000, boot.stratified=TRUE,
                                #	           show.thres=FALSE,
                                print.auc=TRUE,                  # compute AUC (of AUC by default)
                                #                   print.auc.pattern="%.2f",
                                print.thres="best",
                                print.thres.best.method="youden"))

    roc_test<- suppressMessages(roc(meta_data_test$group, meta_data_test$classifier_score,
                                percent=TRUE,
    #                             ci=TRUE, 
                                boot.n=10000, boot.stratified=TRUE,
                                #	           show.thres=FALSE,
                                print.auc=TRUE,                  # compute AUC (of AUC by default)
                                #                   print.auc.pattern="%.2f",
                                print.thres="best",
                                print.thres.best.method="youden"))


    train_mdf <- data.frame("sensitivity" = roc_train$sensitivities,
                            "specificity" = roc_train$specificities,
                        "set" = "train")

    test_mdf <- data.frame("sensitivity" = roc_test$sensitivities,
                            "specificity" = roc_test$specificities,
                        "set" = "test")

 

    mdf_glm <- rbind(train_mdf, test_mdf) %>%
        mutate(FPR = 1-(as.numeric(specificity)/100),
            TPR = as.numeric(sensitivity)/100)
    

    # if(MOD == "bacterial_infection"){
    #     mdf_glm <- test_mdf %>%
    #         mutate(FPR = 1-(as.numeric(specificity)/100),
    #             TPR = as.numeric(sensitivity)/100) %>%
    #         mutate(model = "GLMNET LASSO") %>% mutate(set = "LOO")
    # }

    cat(MOD)
    cat(paste0("\n---> Train AUC: ",as.character(round(roc_train$auc,2))))
    cat(paste0("\n---> Test AUC: ",as.character(round(roc_test$auc,2))))
    cat("\n------------\n")

    ## MAKE PLOT
    auc_plt <- mdf_glm %>% 
        ggplot(aes(x= FPR, y= TPR, group = set, linetype = set, color = set))+
        geom_path()+
        theme_prevail()+
        theme(aspect.ratio = 1,
            axis.title.x = element_blank(),
            axis.text.x = element_text(size = 6),
            axis.title.y = element_blank(), 
            axis.text.y = element_text(size = 6)
            ) +
        scale_linetype_manual(values = c("train"="dashed","test"="solid","LOO"="solid"))+
        # scale_x_continuous(position = "top") +
        scale_color_manual(values = c("train"="blue","test"="red","LOO"="#069547"))

    ## SAVE PLOT
    WIDTH = 1.3
    HEIGHT = 1.3

    pdf(file = paste0("./plots/ONE-ONE_AUC_",MOD,".pdf"),
            width=WIDTH,height=HEIGHT, paper="special", bg="white",
            fonts="Helvetica", colormodel = "srgb", pointsize=6, useDingbats = FALSE)
    print(auc_plt)
    dev.off()
}

KD<>viral_infection
---> Train AUC: 99.28
---> Test AUC: 92.74
------------


MISC<>viral_infection
---> Train AUC: 99.96
---> Test AUC: 96.78
------------
viral_infection<>bacterial_infection
---> Train AUC: 96.92
---> Test AUC: 86.13
------------
MISC<>KD
---> Train AUC: 99.91
---> Test AUC: 98.73
------------
KD<>bacterial_infection
---> Train AUC: 99.29
---> Test AUC: 86.83
------------
MISC<>bacterial_infection
---> Train AUC: 99.89
---> Test AUC: 93.53
------------


### Genes used

In [21]:
comp_genes <- list()
for (comp in names(all_mod)){

    mod_comp <- all_mod[[comp]]$model
    mod_coefs <- coef(mod_comp$finalModel, mod_comp$bestTune$lambda) %>% as.matrix() %>% as.data.frame()
    mod_ngs <- mod_coefs %>% filter(s1 != 0) %>% rownames()
    mod_genes <- mod_ngs[mod_ngs != "(Intercept)"]
    mod_num_genes <- mod_genes %>% length()

    cat(paste0(comp," --> ",as.character(mod_num_genes),"\n"))

    comp_genes[[comp]] <- data.frame("comp"=comp,
                                    "gene"=mod_genes)
}

comp_genes <- do.call("rbind",comp_genes) %>% data.frame() %>% remove_rownames()

comp_genes <- merge(comp_genes, gene_list, by.x="gene",by.y="gene_id") %>% arrange(comp)
comp_genes %>% head()

total_genes <- comp_genes %>% pull(gene) %>% unique() %>% length()

cat(paste0("TOTAL # GENES --> ",as.character(total_genes),"\n"))


### save
comp_genes %>% write.csv("./output/Supplementary-File5_pedInflam.Multi-panel.csv",row.names = F,quote=FALSE)

KD<>bacterial_infection --> 17
KD<>viral_infection --> 29
MISC<>bacterial_infection --> 20
MISC<>viral_infection --> 22
MISC<>KD --> 21
viral_infection<>bacterial_infection --> 17


,gene,comp,gene_name,gene_type
,<chr>,<chr>,<chr>,<chr>
1,ENSG00000029725.17,KD<>bacterial_infection,RABEP1,protein_coding
2,ENSG00000064393.16,KD<>bacterial_infection,HIPK2,protein_coding
3,ENSG00000082641.16,KD<>bacterial_infection,NFE2L1,protein_coding
4,ENSG00000083845.9,KD<>bacterial_infection,RPS5,protein_coding
5,ENSG00000090621.15,KD<>bacterial_infection,PABPC4,protein_coding
6,ENSG00000101752.12,KD<>bacterial_infection,MIB1,protein_coding


TOTAL # GENES --> 109


In [10]:
comp_genes %>% pull(gene_name) %>% unique() %>% paste0(collapse= ", ") %>% cat()

RABEP1, HIPK2, NFE2L1, RPS5, PABPC4, MIB1, BNIP3L, BIRC2, DARS1, AKAP12, LCP1, DST, CTSB, GABARAP, PEAK1, HBA2, MT-TL1, EIF4B, SBNO2, MAP4K4, RASSF2, TDP2, SNX3, HDLBP, WIPF1, ANXA11, HIP1, DNMT1, SH3BP5, KDM6B, PHC2, CD164, HEMGN, MCTP2, PKP4, PNRC1, S100A12, IFI27, GRB2, IFITM1, SVIL, HELZ, SHANK3, H2BC9, VIM, CD44, POU2F2, TCF7, FTL, FXYD5, FKBP5, AKNA, RAPGEF1, PFN1, PHACTR2, TRIR, ATP5F1E, EIF2S2, CGNL1, TRIM28, CA1, EEF2, BCL9L, ANXA6, UBA52, VCAN, LYZ, HSPA8, PTGES3, SUB1, CD48, RPL21, RDX, PRPF8, USF3, NRIP1, MT-TI, TXNIP, CCDC88C, CD74, YBX1, TRIOBP, CSK, GPI, AHNAK, GLUL, FLOT1, HECW2, ARRB2, S100A8, MX1, SLFN5, RBM33, PTMA, BAZ1A, SAMD9, MVP, NUCKS1, HIF1A, LTA4H, CSF3R, TBC1D14, BCL2A1, PGD, TNIP1, SLA, LY6E, DENND5A, S100A4

In [11]:
## ranked list
comp_coefs <- list()
for (comp in names(all_mod)){

    mod_comp <- all_mod[[comp]]$model
    mod_coefs <- coef(mod_comp$finalModel, mod_comp$bestTune$lambda) %>% as.matrix() %>% as.data.frame() %>% rownames_to_column(.,"gene_id")
    mod_coefs$comp <- comp
    comp_coefs[[comp]] <- mod_coefs
}

all_coefs <- do.call("rbind",comp_coefs) %>% 
    data.frame() %>% 
    filter(s1 != 0 ) %>%  filter(gene_id != "(Intercept)") %>%
    mutate(abs_coef = abs(s1)) %>% 
    select(gene_id,abs_coef, comp )


coef_sum <- all_coefs %>% 
    group_by(gene_id) %>% 
    summarise(sum_coef = (abs_coef), count = n()) %>%  
    arrange(desc(count),desc(sum_coef)) %>% 
    mutate(rank = row_number()) #%>% 
    # select(gene_id,rank)

coef_sum %>% head()

coef_sum <- merge(coef_sum, gene_list, by.x="gene_id",by.y="gene_id") %>% arrange(desc(count)) %>% 
    select(gene_name, gene_type, count)
### save
coef_sum %>% write.csv("./output/Supplementary_MultiClass-GenePanel_ranked.csv",row.names = F,quote=FALSE)
# comp_genes %>% write.csv("./output/Supplementary_MultiClass-GenePanel.csv",row.names = F,quote=FALSE)

Warning message:
“Returning more (or less) than 1 row per `summarise()` group was deprecated in
dplyr 1.1.0.
ℹ Please use `reframe()` instead.
ℹ When switching from `summarise()` to `reframe()`, remember that `reframe()`
  always returns an ungrouped data frame and adjust accordingly.”


`summarise()` has grouped output by 'gene_id'. You can override using the
`.groups` argument.


gene_id,sum_coef,count,rank
<chr>,<dbl>,<int>,<int>
ENSG00000131016.17,0.6543236,4,1
ENSG00000131016.17,0.2050490,4,2
ENSG00000131016.17,0.1817550,4,3
ENSG00000131016.17,0.1586897,4,4
ENSG00000130816.17,0.6518683,3,1
ENSG00000130816.17,0.2279840,3,2


In [12]:
res_all <- c()
for (i in 1:nrow(comp_genes)){
    row_tmp <- comp_genes[i,]
    comp_tmp <- row_tmp$comp
    g1 = strsplit(comp_tmp,"<>")[[1]][[1]]
    g2 = strsplit(comp_tmp,"<>")[[1]][[2]]
    gene_tmp <- row_tmp$gene_name

    desq_res <- all_outputs[[g1]][['all_sig_df']] %>% 
        filter(grepl(g2,comp)) %>% 
        filter(gene_name == gene_tmp) %>% 
        pull(log2FoldChange)

    if(desq_res > 0){res_all <- c(res_all, g1)}else{res_all <- c(res_all, g2)}
}

In [13]:
comp_genes_new <- comp_genes
comp_genes_new$elevated <- res_all

In [14]:
comp_genes_new %>% group_by(gene_name, elevated) %>% dplyr::count() %>% arrange(desc(n)) %>% filter(n>1)

gene_name,elevated,n
<chr>,<chr>,<int>
AKAP12,KD,2
AKAP12,MISC,2
BNIP3L,bacterial_infection,2
DNMT1,viral_infection,2
IFI27,viral_infection,2
PABPC4,KD,2
SLFN5,viral_infection,2


---
# Composite Model

### get scores for each sample - model pair

In [3]:
###--------------------------------------------------------------
### Create metadata with all predictions
###--------------------------------------------------------------

res_path = "./ML_split70-30/1_all/output/"
ALL_COMB = list.files(res_path)
ALG = "GLMNETLasso"

meta_all <- rbind(meta_train, meta_test)
counts_all_vst <- cbind(counts_train_vst, counts_test_vst) %>% t() %>% data.frame()

for (GRP in ALL_COMB){
    grp_mod = readRDS(paste0(res_path,GRP,"/",ALG,"/",GRP,".",ALG,".rds"))$model
    tpred = predict(grp_mod, newdata = counts_all_vst, type="prob")
    meta_all[[GRP]] <- as.numeric(tpred[,1])
}

meta_all %>% head

,PTID,sample_id,diagnosis,origin,severity,date_dif,age,inflam_cat,Xsample_id,sample_group_granular,sample_group,cfrna_id,cohort,KD<>bacterial_infection,KD<>viral_infection,MISC<>bacterial_infection,MISC<>KD,MISC<>viral_infection,viral_infection<>bacterial_infection
,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,193100,cfrna_kd_140,KD,UCSD,KD subtype 1,0,2.9,KD,cfrna_kd_140,KD_KD subtype 1_UCSD,KD,cfrna_kd_140,train,0.8905097,0.9807007,0.7201240,0.249445222,0.9560403,0.06587617
2,163045,cfrna_kd_56,KD,UCSD,KD subtype 1,0,1.5,KD,cfrna_kd_56,KD_KD subtype 1_UCSD,KD,cfrna_kd_56,train,0.9704287,0.9806380,0.5164583,0.028376352,0.8484037,0.25301704
3,153002,cfrna_kd_128,KD,UCSD,KD subtype 1,0,8.8,KD,cfrna_kd_128,KD_KD subtype 1_UCSD,KD,cfrna_kd_128,train,0.8367533,0.7269454,0.6691203,0.009904471,0.4408327,0.62770134
4,183003,cfrna_kd_133,KD,UCSD,KD subtype 1,0,8.3,KD,cfrna_kd_133,KD_KD subtype 1_UCSD,KD,cfrna_kd_133,train,0.9757072,0.9920440,0.3896723,0.019276333,0.7681265,0.37091315
5,163014,cfrna_kd_130,KD,UCSD,KD subtype 1,0,6.5,KD,cfrna_kd_130,KD_KD subtype 1_UCSD,KD,cfrna_kd_130,train,0.9012122,0.9341808,0.8126242,0.060262298,0.9895937,0.11740045
6,3511,cfrna_kd_49,KD,UCSD,KD subtype 1,0,4.5,KD,cfrna_kd_49,KD_KD subtype 1_UCSD,KD,cfrna_kd_49,train,0.9630737,0.9975998,0.6089985,0.099099055,0.9617453,0.04122567


### Clustering

In [16]:
## Create data frame for PCA
mdf_cast = meta_all%>%
    filter(inflam_cat %in% c("MISC","KD","viral_infection","Healthy","bacterial_infection")) %>% 
    mutate(inflam_cat = factor(inflam_cat, levels = c("MISC","KD","viral_infection","bacterial_infection","Healthy")))

# select columns for PCA and perform PCA
pca_cols <- ALL_COMB
pca_res <- prcomp(mdf_cast[, pca_cols], scale. = TRUE)

# create a dataframe with PC1 and PC2 scores
pca_df <- mdf_cast
pca_df$PC1 = pca_res$x[,1]
pca_df$PC2 = pca_res$x[,2]
pca_df$PC3 = pca_res$x[,3]
pca_df$PC4 = pca_res$x[,4]

library("plot3D")
options(repr.plot.height = 15, repr.plot.width = 15)

color_mapping <- c(
  MISC = '#b91933',
  KD = '#00aedb',
  viral_infection = '#00b159',
  bacterial_infection = '#ffc425',
  other = '#c97eb3',
  Healthy = '#cccccc'
)

pca_impor <- summary(pca_res)$importance


## figure with legend

WIDTH = 3.4
HEIGHT = 3.4
pdf(file = paste0("./plots/ONE-ONE_PCA.3D.pdf"),
        width=WIDTH,height=HEIGHT, paper="special", bg="white",
        fonts="Helvetica", colormodel = "srgb", pointsize=6, useDingbats = FALSE)

plot3D::scatter3D(x = pca_df$PC1, y = pca_df$PC2, z = pca_df$PC3,
                  colvar = as.numeric(pca_df$inflam_cat), col = INFLAMCAT_FILL_KEY, colkey = FALSE,
                  pch = 16, cex = 1,
                  ticktype = "detailed",
                  bty = "b2",
                  theta = 90, phi = 35,
                  legend = FALSE,
                  # xlab= paste0("PC1"), ylab=paste0("PC2"), zlab=paste("PC3")
                  xlab= paste0("PC1 (",round(pca_impor[2,"PC1"]*100,1),"%)"), 
                  ylab=paste0("PC2 (",round(pca_impor[2,"PC2"]*100,1),"%)"), 
                  zlab=paste0("PC3 (",round(pca_impor[2,"PC3"]*100,1),"%)")
                 # colvar = pca_df$inflam_cat,
                 )

dev.off()

Warning message:
“no DISPLAY variable so Tk is not available”


png 
  2

### train multiclass RF

In [4]:
###--------------------------------------------------------------
### Train multiclass RF
###--------------------------------------------------------------
meta_all_multi <- meta_all %>% 
    filter(inflam_cat %in% c("MISC","KD","viral_infection","bacterial_infection"))  %>% 
    mutate(inflam_cat = factor(inflam_cat, levels = c("MISC","KD","viral_infection","bacterial_infection")))

### Run multiclass random forest

clas_scores = meta_all_multi %>% filter(cohort == "train") %>% select(all_of(c("inflam_cat",ALL_COMB)))

set.seed(42)
control <- trainControl(method="cv",number=5)
metric <- "Accuracy"
tunegrid <- expand.grid(.mtry = (1:length(ALL_COMB)))

rf_fit <- train(inflam_cat~., 
                        data=clas_scores, 
                        method='rf', 
                        metric=metric, 
                    # preProcess = c("center","scale"),
                        tuneGrid = tunegrid, 
                        trControl=control)


# Make predictions on the test set
model_probs <- predict(rf_fit, newdata = meta_all_multi,type="prob") 
meta_all_multi$predictions <- predict(rf_fit, newdata = meta_all_multi) 

meta_all_multi <- cbind(meta_all_multi, model_probs)

In [5]:
model_probs$cfrna_id <- meta_all_multi$cfrna_id
model_probs%>% write.csv("output/all_multiclass_scores.csv",quote=FALSE, row.names=FALSE)

meta_all_multi %>% 
    select(sample_id,PTID, cohort, age, origin, inflam_cat, predictions) %>% 
    write.csv("./output/all_multiclass_predictions.csv",quote=FALSE, row.names=FALSE)

### get results

In [6]:
res_train <- meta_all_multi %>% filter(cohort == "train")
conf_matrix <- confusionMatrix(res_train$predictions, res_train$inflam_cat)
print(conf_matrix)

Confusion Matrix and Statistics

                     Reference
Prediction            MISC KD viral_infection bacterial_infection
  MISC                  66  0               0                   0
  KD                     0 68               0                   0
  viral_infection        0  0              71                   0
  bacterial_infection    0  0               0                  27

Overall Statistics
                                     
               Accuracy : 1          
                 95% CI : (0.9842, 1)
    No Information Rate : 0.306      
    P-Value [Acc > NIR] : < 2.2e-16  
                                     
                  Kappa : 1          
                                     
 Mcnemar's Test P-Value : NA         

Statistics by Class:

                     Class: MISC Class: KD Class: viral_infection
Sensitivity               1.0000    1.0000                  1.000
Specificity               1.0000    1.0000                  1.000
Pos Pred Value         

In [7]:
# res_train <- meta_all_multi %>% filter(cohort == "train")
# conf_matrix <- confusionMatrix(res_train$predictions, res_train$inflam_cat)
# print(conf_matrix)

res_test <- meta_all_multi %>% filter(cohort == "test")
conf_matrix <- confusionMatrix(res_test$predictions, res_test$inflam_cat)
print(conf_matrix)

Confusion Matrix and Statistics

                     Reference
Prediction            MISC KD viral_infection bacterial_infection
  MISC                  29  2               0                   1
  KD                     0 27               4                   4
  viral_infection        1  2              28                   3
  bacterial_infection    1  2               2                   6

Overall Statistics
                                          
               Accuracy : 0.8036          
                 95% CI : (0.7178, 0.8726)
    No Information Rate : 0.3036          
    P-Value [Acc > NIR] : <2e-16          
                                          
                  Kappa : 0.7288          
                                          
 Mcnemar's Test P-Value : 0.6049          

Statistics by Class:

                     Class: MISC Class: KD Class: viral_infection
Sensitivity               0.9355    0.8182                 0.8235
Specificity               0.9630    0.8987  

In [10]:
GROUP = "KD"
meta_all_multi %>% 
    filter(cohort == "test") %>% 
    mutate(GRP = ifelse(inflam_cat == GROUP,GROUP,"Other")) %>% 
    mutate(RES = ifelse(GRP == GROUP & predictions == GROUP,"TP",
                        ifelse(GRP == GROUP & predictions != GROUP,"FP",
                            ifelse(GRP != GROUP & predictions == GROUP,"FN","TN")))) %>% 
    pull(RES) %>% table()

(71 + 27) / (71 + 27 + 8 + 6)

.
FN FP TN TP 
 8  6 71 27 

[1] 0.875

In [9]:
meta_all_multi %>% filter(predictions == inflam_cat) %>% 
    select(PTID, diagnosis, cohort, origin) %>% 
    arrange(origin) %>% 
    write.csv("./Patient-ids_vignettes_2024.01.18.csv",row.names=FALSE,quote=FALSE)

### Plot confusion matrix

In [10]:
### extract train and test
mdf_cast_train <- meta_all_multi %>% filter(cohort == "train")
mdf_cast_test <- meta_all_multi %>% filter(cohort == "test")

conf_trn <- confusionMatrix(mdf_cast_train$predictions, mdf_cast_train$inflam_cat)
conf_tst <- confusionMatrix(mdf_cast_test$predictions, mdf_cast_test$inflam_cat)

conf_trn$overall['Accuracy']
conf_tst$overall['Accuracy']

as.data.frame(as.table(conf_trn)) %>%  head()

Accuracy 
       1

Accuracy 
0.8035714

,Prediction,Reference,Freq
,<fct>,<fct>,<int>
1,MISC,MISC,66
2,KD,MISC,0
3,viral_infection,MISC,0
4,bacterial_infection,MISC,0
5,MISC,KD,0
6,KD,KD,68


In [14]:
mdf_cast_train <- meta_all_multi %>% filter(cohort == "train")
mdf_cast_test <- meta_all_multi %>% filter(cohort == "test")

conf_trn <- confusionMatrix(mdf_cast_train$predictions, mdf_cast_train$inflam_cat)
conf_tst <- confusionMatrix(mdf_cast_test$predictions, mdf_cast_test$inflam_cat)

trn_count <- mdf_cast_train %>% group_by(inflam_cat) %>% count() %>% rename(Reference = inflam_cat)
conf_trn <- as.data.frame(as.table(conf_trn))
conf_trn <- merge(conf_trn,trn_count, by="Reference", all.x = TRUE) %>% mutate(FRACTION = Freq/n)

conf_trn %>% head()

,Reference,Prediction,Freq,n,FRACTION
,<fct>,<fct>,<int>,<int>,<dbl>
1,bacterial_infection,MISC,0,27,0
2,bacterial_infection,KD,0,27,0
3,bacterial_infection,viral_infection,0,27,0
4,bacterial_infection,bacterial_infection,27,27,1
5,KD,MISC,0,68,0
6,KD,KD,68,68,1


In [20]:
### extract train and test
mdf_cast_train <- meta_all_multi %>% filter(cohort == "train")
mdf_cast_test <- meta_all_multi %>% filter(cohort == "test")

conf_trn <- confusionMatrix(mdf_cast_train$predictions, mdf_cast_train$inflam_cat)
conf_tst <- confusionMatrix(mdf_cast_test$predictions, mdf_cast_test$inflam_cat)

conf_trn$overall['Accuracy']
conf_tst$overall['Accuracy']


trn_count <- mdf_cast_train %>% group_by(inflam_cat) %>% count() %>% rename(Reference = inflam_cat)
conf_trn <- as.data.frame(as.table(conf_trn))
conf_trn <- merge(conf_trn,trn_count, by="Reference", all.x = TRUE) %>% mutate(FRACTION = Freq/n)

tst_count <- mdf_cast_test %>% group_by(inflam_cat) %>% count() %>% rename(Reference = inflam_cat)
conf_tst <- as.data.frame(as.table(conf_tst))
conf_tst <- merge(conf_tst,tst_count, by="Reference", all.x = TRUE) %>% mutate(FRACTION = Freq/n)



Accuracy 
       1

Accuracy 
0.8035714

In [23]:
### extract train and test
mdf_cast_train <- meta_all_multi %>% filter(cohort == "train")
mdf_cast_test <- meta_all_multi %>% filter(cohort == "test")

conf_trn <- confusionMatrix(mdf_cast_train$predictions, mdf_cast_train$inflam_cat)
conf_tst <- confusionMatrix(mdf_cast_test$predictions, mdf_cast_test$inflam_cat)

conf_trn$overall['Accuracy']
conf_tst$overall['Accuracy']



trn_count <- mdf_cast_train %>% group_by(inflam_cat) %>% count() %>% rename(Reference = inflam_cat)
conf_trn <- as.data.frame(as.table(conf_trn))
conf_trn <- merge(conf_trn,trn_count, by="Reference", all.x = TRUE) %>% mutate(FRACTION = Freq/n)

tst_count <- mdf_cast_test %>% group_by(inflam_cat) %>% count() %>% rename(Reference = inflam_cat)
conf_tst <- as.data.frame(as.table(conf_tst))
conf_tst <- merge(conf_tst,tst_count, by="Reference", all.x = TRUE) %>% mutate(FRACTION = Freq/n)



# Plot the confusion matrix using ggplot2
table_train_plt <- conf_trn %>% 
  ggplot(aes(x = Reference, y = Prediction, fill = FRACTION)) +
    geom_tile(color = "white") +
    scale_fill_gradientn(colors = c("#FAD02E", "#F28D35", "#D33F6A", "#2A295A"), na.value = "white") +
    geom_text(aes(label = sprintf("%d", Freq)), vjust = 1, size = 2) +
    theme_minimal() +
    theme(
      axis.text = element_text(size = 6),
      axis.title = element_blank(),
      plot.title = element_blank(),
      legend.position = "none")

table_test_plt <- conf_tst %>% 
  ggplot(aes(x = Reference, y = Prediction, fill = FRACTION)) +
    geom_tile(color = "white") +
    scale_fill_gradientn(colors = c("#FAD02E", "#F28D35", "#D33F6A", "#2A295A"), na.value = "white") +
    # scale_fill_gradientn(colors = c("#FCEE21", "#009245"), na.value = "white") +
    # scale_fill_gradientn(colors = c("#c4e0e5", "#4ca1af"), na.value = "white") +
    geom_text(aes(label = sprintf("%d", Freq)), vjust = 1, size = 2, color = "black") +
    theme_minimal() +
    theme(
      axis.text = element_text(size = 6),
      axis.title = element_blank(),
      plot.title = element_blank(),
      legend.position = "none")


### SAVE PLOTS
WIDTH = 2.5
HEIGHT = 1.8

pdf(file = paste0("./plots/ONE-ONE_confusionMtrx-TRAIN.pdf"),
        width=WIDTH,height=HEIGHT, paper="special", bg="white",
        fonts="Helvetica", colormodel = "srgb", pointsize=6, useDingbats = FALSE)
table_train_plt
dev.off()

pdf(file = paste0("./plots/ONE-ONE_confusionMtrx-TEST.pdf"),
        width=WIDTH,height=HEIGHT, paper="special", bg="white",
        fonts="Helvetica", colormodel = "srgb", pointsize=6, useDingbats = FALSE)
table_test_plt
dev.off()



Accuracy 
       1

Accuracy 
0.8035714

png 
  2

png 
  2

---
### Metrics for each comparison

In [6]:
### 1 vs 1 models


########################################
### 1 vs 1 models
########################################
res_path = "./ML_split70-30/1_all/output/"

all_comps = c(
    "KD<>bacterial_infection",
    "KD<>viral_infection",
    "MISC<>bacterial_infection",
    "MISC<>viral_infection",
    "MISC<>KD",
    "viral_infection<>bacterial_infection"
)

ALG = "GLMNETLasso"
all_stats_1v1 <- list()

for (GRP in all_comps){
    train_stats <- read.delim(paste0(res_path,GRP,"/",ALG,"/",GRP,".",ALG,".train.txt")) %>% t() %>% data.frame() %>%  select(Sensitivity, Specificity,Accuracy, auc) %>%
        mutate(cohort = "train", comp = GRP)
    test_stats <- read.delim(paste0(res_path,GRP,"/",ALG,"/",GRP,".",ALG,".test.txt"))%>% t() %>% data.frame() %>%  select(Sensitivity, Specificity,Accuracy, auc) %>%
        mutate(cohort = "test", comp = GRP)
    all_stats_1v1[[GRP]] <- rbind(train_stats, test_stats)
}
all_stats_1v1 <- do.call(rbind,all_stats_1v1) %>% data.frame() %>% remove_rownames()

In [9]:
all_stats_1v1 %>% filter(cohort=="test") %>% arrange(desc(Accuracy)) %>% tail()

,Sensitivity,Specificity,Accuracy,auc,cohort,comp
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,0.90625,0.9375,0.921875,0.9873046875,test,MISC<>KD
2,0.84375,0.941176470588235,0.893939393939394,0.967830882352941,test,MISC<>viral_infection
3,0.96875,0.714285714285714,0.891304347826087,0.868303571428571,test,KD<>bacterial_infection
4,0.90625,0.785714285714286,0.869565217391304,0.935267857142857,test,MISC<>bacterial_infection
5,0.9375,0.794117647058823,0.863636363636364,0.927389705882353,test,KD<>viral_infection
6,0.882352941176471,0.714285714285714,0.833333333333333,0.861344537815126,test,viral_infection<>bacterial_infection


In [80]:
### 1 vs 1 models


########################################
### 1 vs 1 models
########################################
res_path = "./ML_split70-30/1_all/output/"

all_comps = c(
    "KD<>bacterial_infection",
    "KD<>viral_infection",
    "MISC<>bacterial_infection",
    "MISC<>viral_infection",
    "MISC<>KD",
    "viral_infection<>bacterial_infection"
)

ALG = "GLMNETLasso"
all_stats_1v1 <- list()

for (GRP in all_comps){
    train_stats <- read.delim(paste0(res_path,GRP,"/",ALG,"/",GRP,".",ALG,".train.txt")) %>% t() %>% data.frame() %>%  select(Sensitivity, Specificity, auc) %>%
        mutate(cohort = "train", comp = GRP)
    test_stats <- read.delim(paste0(res_path,GRP,"/",ALG,"/",GRP,".",ALG,".test.txt"))%>% t() %>% data.frame() %>%  select(Sensitivity, Specificity, auc) %>%
        mutate(cohort = "test", comp = GRP)
    all_stats_1v1[[GRP]] <- rbind(train_stats, test_stats)
}
all_stats_1v1 <- do.call(rbind,all_stats_1v1) %>% data.frame() %>% remove_rownames() %>% select(comp, cohort, auc, Sensitivity, Specificity) %>% 
        rename(Comparison = comp, Cohort = cohort, AUC = auc) %>% 
        mutate(Sensitivity = round(as.numeric(Sensitivity),2),
                Specificity = round(as.numeric(Specificity),2),
                AUC = round(as.numeric(AUC),2)) %>% 
        mutate(Comparison = gsub("<>"," vs ",Comparison)) %>% 
        mutate(Comparison = gsub("_"," ",Comparison))%>% 
        mutate(Comparison = gsub("MISC","MIS-C",Comparison))

########################################
### 1 vs all model
########################################

all_GRPS = c("MISC","KD","viral_infection","bacterial_infection")

res_train <- meta_all_multi %>% filter(cohort == "train")
res_test <- meta_all_multi %>% filter(cohort == "test")

conf_matrix_train <- confusionMatrix(res_train$predictions, res_train$inflam_cat)$byClass %>% data.frame() %>% 
    select(Sensitivity, Specificity) %>% 
    rownames_to_column("Comparison") %>% 
    mutate(Comparison = paste0(gsub("Class: ","",Comparison)," vs All")) %>% 
    mutate(Cohort = "train", AUC = NA) %>% 
    select(Comparison, Cohort, AUC, Sensitivity, Specificity)
conf_matrix_test <- confusionMatrix(res_test$predictions, res_test$inflam_cat)$byClass %>% data.frame() %>% 
    select(Sensitivity, Specificity)%>% 
    rownames_to_column("Comparison") %>% 
    mutate(Comparison = paste0(gsub("Class: ","",Comparison)," vs All")) %>% 
    mutate(Cohort = "test", AUC = NA) %>% 
    select(Comparison, Cohort, AUC, Sensitivity, Specificity)

all_stats_1vall <- rbind(conf_matrix_train,conf_matrix_test) %>% data.frame() %>% 
        mutate(Sensitivity = round(as.numeric(Sensitivity),2),
                Specificity = round(as.numeric(Specificity),2),
                AUC = round(as.numeric(AUC),2)) %>% 
        mutate(Comparison = gsub("MISC","MIS-C",Comparison))%>% 
        arrange(Comparison, desc(Cohort))


########################################
### Combine
########################################
all_stats_all <- rbind(all_stats_1v1,all_stats_1vall)

all_stats_all %>% write.csv("./output/Table2_AllStats.csv",row.names=F, quote=FALSE)

In [79]:
all_stats_1vall

Comparison,Cohort,AUC,Sensitivity,Specificity
<chr>,<chr>,<dbl>,<dbl>,<dbl>
KD vs All,train,NA,1.00,1.00
KD vs All,test,NA,0.84,0.90
MIS-C vs All,train,NA,1.00,1.00
MIS-C vs All,test,NA,0.91,0.96
bacterial_infection vs All,train,NA,1.00,1.00
bacterial_infection vs All,test,NA,0.43,0.95
viral_infection vs All,train,NA,1.00,1.00
viral_infection vs All,test,NA,0.82,0.92


In [74]:
all_stats_1v1 %>% head()
all_stats_1vall %>% head()


,Comparison,Cohort,AUC,Sensitivity,Specificity
,<chr>,<chr>,<dbl>,<dbl>,<dbl>
1,KD vs bacterial infection,train,0.99,0.94,1.00
2,KD vs bacterial infection,test,0.87,0.97,0.71
3,KD vs viral infection,train,0.99,0.97,0.96
4,KD vs viral infection,test,0.93,0.94,0.79
5,MIS-C vs bacterial infection,train,1.00,0.98,1.00
6,MIS-C vs bacterial infection,test,0.94,0.91,0.79


,Comparison,cohort,AUC,Sensitivity,Specificity
,<chr>,<chr>,<lgl>,<dbl>,<dbl>
1,KD vs All,train,NA,1.0000000,1.0000000
2,KD vs All,test,NA,0.8437500,0.9000000
3,MISC vs All,train,NA,1.0000000,1.0000000
4,MISC vs All,test,NA,0.9062500,0.9625000
5,bacterial_infection vs All,train,NA,1.0000000,1.0000000
6,bacterial_infection vs All,test,NA,0.4285714,0.9489796
